In [1]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import SAC, TD3, A2C
import matplotlib.pyplot as plt
import pickle
import os
import argparse
import asyncio

In [2]:
models_dir = 'models'
logs_dir = 'logs'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

In [3]:
def train(env, sb3_algo, max_iters=20, model=None, curr_name=None, gamma=0.99):
    if model is None:
        match sb3_algo:
            case 'SAC':
                model = SAC('MlpPolicy', env, verbose=1, tensorboard_log=logs_dir, gamma=gamma)
            case 'TD3':
                model = TD3('MlpPolicy', env, verbose=1, tensorboard_log=logs_dir, gamma=gamma)
            case 'A2C':
                model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=logs_dir, gamma=gamma)
            case _:
                print('Invalid algorithm')
                return
        name = f'{models_dir}/{sb3_algo}'
    else:
        if curr_name is None:
            print('Please provide a name for the model')
            return
        name = curr_name
        model.set_env(env)

    TIMESTEPS = 25000
    iters = 0
    while True and iters < max_iters:
        iters +=1
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False)
        model.save(f'{name}_{TIMESTEPS*iters}')

In [4]:
gymenv = gym.make('Humanoid-v4', render_mode=None)
# train(gymenv, 'SAC')
# train(gymenv, 'TD3')
# train(gymenv, 'A2C')

In [5]:
def test(env, sb3_algo, path_to_model):
    match sb3_algo:
        case 'SAC':
            model = SAC.load(path_to_model)
        case 'TD3':
            model = TD3.load(path_to_model)
        case 'A2C':
            model = A2C.load(path_to_model)
        case _:
            print('Invalid algorithm')
            return

    obs = env.reset()[0]
    done = False
    extra_steps = 500
    while True:
        action, _states = model.predict(obs)
        obs, _, done, _, _ = env.step(action)
        
        if done:
            extra_steps -= 1

        if extra_steps < 0:
            break

In [6]:
gymenv_test = gym.make('Humanoid-v4', render_mode='human')

na podstawie tenorboarda stwierdzamy ze sac jest najlepszy

In [7]:
test(gymenv_test, 'SAC', 'models/SAC3_continued_400000')

Pouczymy model jescze przez kolejne 100000 epok, zaczynajac od poprzedniego stanu

In [8]:
model = SAC.load('models/SAC2_continued_100000')
gymenv = gym.make('Humanoid-v4', render_mode=None)
# train(gymenv, 'SAC', model=model, curr_name='models/SAC3_continued')

/Users/szymon/miniconda3/envs/IOwADC/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(
/Users/szymon/miniconda3/envs/IOwADC/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 6.09GB > 5.43GB
  warnings.warn(


In [9]:
# test(gymenv_test, 'SAC', 'models/SAC2_continued_100000')

: 